# Implementação da Rede Neural Convolucional - CNN

Alunos: Ana Carolina da Silva, Gustavo Baroni Bruder, Luiz Gustavo Klitzke, Rodrigo Kapulka Franco, Nicole Bauchspiess

In [7]:
import os

import cv2 as cv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow import keras

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

## Carregar o <i>dataset</i> a partir da estrutura de pastas

In [8]:
nomes_classes = ["rotten", "healthy"]

def load_data_path(path, class_type, x, y):
    for img_path in os.listdir(path):
        img = cv.imread(f'{path}/{img_path}', cv.IMREAD_GRAYSCALE)
        x.append(cv.resize(img, (224, 224)) / 255.0)
        y.append(class_type)

def load_dataset():
    x_train = []
    y_train = []
    x_test = []
    y_test = []

    load_data_path("data/train/Banana__Healthy", 1, x_train, y_train)
    load_data_path("data/train/Banana__Rotten", 0, x_train, y_train)
    load_data_path("data/test/Banana__Healthy", 1, x_test, y_test)
    load_data_path("data/test/Banana__Rotten", 0, x_test, y_test)

    return np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)

x_train, y_train, x_test, y_test = load_dataset()

## Normalização dos dados

In [9]:
X_train_new = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
X_test_new = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

## Configurar a CNN

In [10]:
cnn = keras.models.Sequential([
        keras.layers.InputLayer(input_shape=(224, 224, 1)),

        keras.layers.Conv2D(64, 5, activation='relu', padding='same', kernel_initializer='glorot_uniform'),
        keras.layers.MaxPool2D(2),

        keras.layers.Conv2D(128, 3, activation='relu', padding='same'),

        keras.layers.Conv2D(128, 3, activation='relu', padding='same'),

        keras.layers.MaxPool2D(2),

        keras.layers.Conv2D(258, 3, activation='relu', padding='same'),

        keras.layers.Conv2D(258, 3, activation='relu', padding='same'),
        keras.layers.MaxPool2D(2),

        keras.layers.Flatten(),

        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dropout(rate=0.5),

        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dropout(rate=0.5),
        
        keras.layers.Dense(2, activation="softmax")])

cnn.summary()

c:\Users\luizk\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 224, 224, 64)   │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 56, 56, 258)    │       297,474 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 56, 56, 258)    │       599,334 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 28, 28, 258)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 202272)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │    25,890,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,019,242 (103.07 MB)

 Trainable params: 27,019,242 (103.07 MB)

 Non-trainable params: 0 (0.00 B)

## Treinar a rede

In [ ]:
cnn.compile(loss="sparse_categorical_crossentropy",
               optimizer="sgd",
               metrics=["accuracy"])
history_nn = cnn.fit(X_train_new, y_train, epochs=20, validation_data=(X_test_new, y_test))

## Salvar e/ou carregar dados de treinamento para uso posterior

In [ ]:
cnn.save("resultados/cnn.keras")

In [ ]:
cnn = keras.models.load_model("resultados/cnn.keras")

## Avaliar os resultados do treinamento

In [ ]:
cnn.evaluate(X_test_new, y_test, verbose = 1)

In [ ]:
pd.DataFrame(history_nn.history).plot(figsize=(12, 8))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

### Matrizes de confusão de algumas partes do dataset

In [12]:
def monta_matriz_confusao(labels_reais, labels_estimadas, titulo):
    cm = confusion_matrix(labels_reais, labels_estimadas)
    disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = nomes_classes)

    # Plotar a matriz de confusão
    plt.figure(figsize = (8, 8))
    disp.plot(cmap = plt.cm.Blues)
    plt.title(titulo)
    plt.xlabel("Classe real")
    plt.ylabel("Classe estimada")
    plt.xticks(rotation =  45)

In [ ]:
result = cnn.predict(x_test[:10])

rotulos_esperados_primeiros_10 = []
rotulos_obtidos_primeiros_10 = []

for i in range(len(result)):
    rotulos_obtidos_primeiros_10.append(np.argmax(result[i], axis=-1))
    rotulos_esperados_primeiros_10.append(y_test[i])
    print(f'Previsao: {np.argmax(result[i], axis=-1)} | Verdadeiro: {y_test[i]}')

monta_matriz_confusao(rotulos_esperados_primeiros_10, rotulos_obtidos_primeiros_10, "Matriz Confusão - Primeiros 10")

In [ ]:
result = cnn.predict(x_test[200:])

rotulos_esperados_apos_200 = []
rotulos_obtidos_apos_200 = []

for i in range(len(result)):
    rotulos_obtidos_apos_200.append(np.argmax(result[i], axis=-1))
    rotulos_esperados_apos_200.append(y_test[200 + i])
    print(f'Previsao: {np.argmax(result[i], axis=-1)} | Verdadeiro: {y_test[200 + i]}')

monta_matriz_confusao(rotulos_esperados_apos_200, rotulos_obtidos_apos_200, "Matriz Confusão - Após 200")